In [1]:
import torch as t
import torch.nn as nn
import einops
from jaxtyping import Int, Float
from typing import List, Optional, Tuple
from transformer_lens import HookedTransformer
import circuitsvis as cv
from circuitsvis.attention import attention_patterns, attention_pattern
from InductionHeadMethods import InductionTask as IT
import plotly.express as px
from InductionHeadMethods import InductionHook

t.set_grad_enabled(False)
device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [2]:
model: HookedTransformer = HookedTransformer.from_pretrained("gpt2-small")

C:\Users\614987704\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [3]:
tokens, logits, cache = IT.run_and_cache_model_repeated_tokens(model, 50)
cache.remove_batch_dim()
str_tokens = model.to_str_tokens(tokens)

In [4]:
rep_tokens_10 = IT.generate_repeated_tokens(model, 50, 10)

In [5]:
induction_hook = InductionHook(model)

model.run_with_hooks(
    rep_tokens_10,
    return_type=None,
    fwd_hooks=[
        (induction_hook.filter, induction_hook.hook)
    ]
)

In [6]:
px.imshow(
    induction_hook.score_store,
    labels={"x": "Head", "y": "Layer"},
    title="Induction Score by Head",
    text_auto=".2f",
    width=900, height=400
)